<a href="https://colab.research.google.com/github/MartinHeHeHe/generalAlgorithms/blob/main/gpt2pipelinestructure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers
!pip install -U PyPDF2
!pip install python-docx
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.4/237.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx
import requests
from bs4 import BeautifulSoup

In [3]:
#Need to remove poznamky redakce

url = "https://zpravy.aktualne.cz/domaci/rozhovor-petr-fiala/r~22e91baa4e4011eea3c0ac1f6b220ee8/"

# Send an HTTP GET request to the URL and fetch the page content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the <div> element with class 'article__content' and id 'article-content'
    article_content = soup.find('div', {'class': 'article__content', 'id': 'article-content'})

    # Initialize variables to store questions and answers
    questions = []
    answers = []
    current_question = None
    current_answer = ""

    # Iterate through elements within 'article-content'
    for element in article_content:
        if element.name == 'h4':

            # New Question
            if current_question is not None:
                questions.append(current_question)
                answers.append(current_answer)
            current_question = element.get_text()
            current_answer = ""  # Reset the answer for the new question
        elif element.name == 'p' and current_question is not None:
            # Answer
            answer_text = element.get_text()
            current_answer += answer_text + ' '  # Concatenate answers into one string

    if current_question is not None:
        questions.append(current_question)
        answers.append(current_answer)

    file_name = "./data/questions_and_answers.txt"

    # Open the file in write mode and write the questions and answers
    with open(file_name, "w") as file:
        for question, answer in zip(questions, answers):
            file.write("Question: " + question + "\n")
            file.write("Answer: " + answer + "\n\n")
else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)


In [4]:
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        combined_text += read_txt(file_path)
    return combined_text

In [5]:
train_directory = './data'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

In [6]:
print(text_data)

Question: Když slyším, jak mluvíte v různých rozhovorech, přemýšlím nad tím, zda vnímáte reálný stav naší země. Nejste optimista jen navenek?
Answer: Vždycky jsem vám říkal, že vím, co dělám, že se držím své strategie a že volby vyhrajeme. Nevěřil jste mi stejně jako mnozí jiní. A my jsme volby vyhráli. Takže jestli se mě nyní ptáte, jestli vnímám realitu a svá slova myslím opravdu vážně, tak ano. Vnímám realitu, analyzuji ji, řídím se jí a poslouchám lidi. Můj optimismus není optimismus, ale realismus. Tak, jak mi to vyšlo dříve, to vyjde i teď.  
Question: Vnímáte ale, že dnes i mnozí vaši voliči mají pocit, že se vládě nedaří tak, jak si to představovali? Důvěřuje vám podle posledních měření jen čtvrtina populace. 
Answer: Kritické období jsme přečkali velmi dobře a teď máme všechny předpoklady k tomu, aby se začala spravovat ekonomika a my si poradili s inflací. A je to i díky tomu, že jsme rekordně investovali do dopravní infrastruktury, nezapomněli jsme na vzdělávání a řešili jsm

In [7]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [8]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [9]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [10]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
  trainer.train()
  trainer.save_model()

In [11]:
train_file_path = "./data/questions_and_answers.txt"
model_name = 'gpt2'
output_dir = './model'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50.0
save_steps = 50000

In [12]:
import accelerate

accelerate.__version__

'0.24.0'

In [13]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


Step,Training Loss


In [14]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [15]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [21]:
model2_path = "./model"
sequence2 = "Question:"
max_len = 50
generate_text(model2_path, sequence2, max_len)

Question: A moje vás a duchem velho zástupcem úmniž, která se řekli vizi problémů většímá,


In [ ]:
# https://212digital.medium.com/fine-tuning-the-gpt-2-large-language-model-unlocking-its-full-potential-66e3a082ab9c
